In [1]:
%cd ..

/data/ephemeral/gamchan/level2-nlp-generationfornlp-nlp-06-lv3


/data/ephemeral/gamchan/level2-nlp-generationfornlp-nlp-06-lv3/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
import os

import pandas as pd
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from tqdm import tqdm
from tqdm.asyncio import tqdm

from prompts import load_template
from utils import DataVersionManager

/data/ephemeral/gamchan/level2-nlp-generationfornlp-nlp-06-lv3/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
load_dotenv()
dvm = DataVersionManager()

PARAGRAPH = "paragraph"
QUESTION = "question"
CHOICES = "choices"
QUESTION_PLUS = "question_plus"
ANSWER = "answer"
REASONING = "reasoning"

In [ ]:
# 현재 실험은 3.1.x 입니다.
df = dvm.search_experiments_integration_data(is_experiment=False)
df.head()

In [ ]:
prompt = PromptTemplate.from_template(
    template=load_template(file_name="page_summarization.txt", template_type="paragraph_generation")
)

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=os.getenv("OPENAI_API_KEY"),
)

summarizer = prompt | llm

In [ ]:
def summarize(row: pd.Series) -> str:
    """
    위키피디아 문서를 요약하는 함수

    Args:
        row (pd.Series): 제공된 데이터셋의 한 행(row), 문제 하나

    Returns:
        str: 추출된 5개 키워드
    """

    keywords_list = [row[f"keyword_{idx}"] for idx in range(1, 6)]
    keywords = ", ".join(keywords_list)
    documents = "\n".join([row[f"keyword_{i}_page"] for i in range(1, 6)])

    response = summarizer.invoke(
        {
            "keywords": keywords,
            "documents": documents,
        },
    )
    return response.content


tqdm.pandas()
sampled_df = df.sample(20, random_state=1004)
sampled_df[f"summarized_wikipedia"] = sampled_df.progress_apply(lambda row: summarize(row), axis=1)

sampled_df.head()

In [ ]:
# 현재 실험은 3.1.x 입니다.
sampled_df.to_csv(dvm.update_file_path(3, 1, save_in_experiment=True), index=False)